In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from lxml import html
import datetime
import time
import selenium
from selenium import webdriver
from urllib2 import urlopen
import requests
import unidecode
import schedule
import os

In [2]:
def scroll(driver):
    driver.execute_script(""" 
        (function () { 
        
            var y = document.body.scrollTop; 
            var step = 100; 
            window.scroll(0, y); 
            function f() { 
                if (y < document.body.scrollHeight) { 
                    y += step; 
                    window.scroll(0, y); 
                    setTimeout(f, 120); 
                }
                else { 
                    window.scroll(0, y); 
                    document.title += "scroll-done"; 
                } 
            } 
            setTimeout(f, 1000); 
        })(); 
        """)

In [3]:
def open_driver(path=r'D:\GA\chromedriver.exe'):
    global driver
    driver = webdriver.Chrome(path) #chromedriver may need to be upgraded
    driver.set_page_load_timeout(120)
    driver.get('https://castbox.fm/categories/0?country=us')
    scroll(driver)


In [4]:
def get_soup(driver):

    html=driver.page_source
    soup=BeautifulSoup(html,'lxml')
    return soup
    

In [5]:
def get_links(soup):
    global linklst
    linklst=[]
    for i in soup.find_all('div',class_ = 'bottomCon clearfix'):
        temp = str('https://castbox.fm') + str(i.a.get('href'))
        linklst.append(temp)
    print len(linklst)
    %store linklst
    return linklst

In [6]:
def get_title(soup):

    title=soup.find('div',class_= 'ch_feed_info').find('h1').get('title')
    return title


In [7]:
def get_info(soup):

    temp=soup.find('div',class_='channelInfo')
    author=temp.find('p',class_='author').text.split(': ')
    subscribed=temp.find('span',class_='count sub_count').text.split(': ')
    played=temp.find('span',class_='count play_count').text.split(': ')
    return author[1],subscribed[1],played[1]

In [8]:
def get_num_epi(soup):

    temp=soup.find('div',class_='trackListCon_title').text
    temp=unidecode.unidecode(temp).split(' ')
    return temp[0]

In [9]:
def get_num_comment(soup):

    temp=soup.find('div',class_='commentList-title').text
    temp=unidecode.unidecode(temp).split(' ')
    return temp[1].strip('()')


In [10]:
def get_desc(soup):

    temp=soup.find('div',class_='des-con').text
    temp=unidecode.unidecode(temp)
    return temp

In [11]:
def get_last_epi(soup):

    temp=soup.find('div',class_='trackRowBig-con')
    title=temp.find('p').get('title')
    temp2=temp.find('p',class_='trackRowBig-con-des')
    date=temp2.find('span',class_='item icon date').text
    dura=temp2.find('span',class_='item icon time').text
    heart=temp2.find_all('span',class_='item')[-1].text
    return title,date,dura,heart
    

In [12]:
def download(linklst,result):
    loopcount = 0
    for link in linklst:
        r=requests.get(link)
        soup = BeautifulSoup(r.content,'lxml')

        title = get_title(soup)
        author,subscribe,play = get_info(soup)
        num_epi = get_num_epi(soup)
        num_comment = get_num_comment(soup)
        desc = get_desc(soup)
        title_,date_,dura_,heart_=get_last_epi(soup)

        idx=str(loopcount)
        pos = str(loopcount + 1)
        lk = str(linklst[loopcount])
        today=datetime.datetime.today().strftime('%Y-%m-%d')

        temp = [idx,title, ' ' ,author,desc,lk,today,pos,num_comment,num_epi,subscribe,play,title_,date_,dura_,heart_]
        d = pd.DataFrame([temp],columns=['Index','Name','Release Date','Author','Description','URL','Date','Position','Comments',
                                   'Total Episodes','Subscribed','Played','Latest Episode Name','Latest Episode Date',
                                   'Duration','Latest Episode Hearts'])
        result = result.append(d)
        print 'Show ranking',loopcount+1,'finished'
        if loopcount%10 == 0:
            time.sleep(5)
        loopcount += 1
    return result

In [13]:
def assignment():
    print "\tStart saving "+str(datetime.datetime.today().strftime('%m%d'))+"\'s data...'"
    open_driver()
    time.sleep(60) # waiting for selenium to scroll down to bottom
    soup=get_soup(driver)
    linklst=get_links(soup)
    
    result = pd.DataFrame(columns=['Index','Name','Release Date','Author','Description','URL','Date','Position','Comments',
                               'Total Episodes','Subscribed','Played','Latest Episode Name','Latest Episode Date',
                               'Duration','Latest Episode Hearts'])
    
    final_result=download(linklst,result)
    os.chdir('D:\GA\Data')
    fileName = 'Castbox ranking '+str(datetime.datetime.today().strftime('%m%d')) +'.csv'
    final_result.to_csv(fileName, encoding='utf8')
    print "\tSaved successfully! See you tomorrow!"
    


In [14]:
schedule.every().day.at("11:53").do(assignment)

Every 1 day at 11:53:00 do assignment() (last run: [never], next run: 2019-03-29 11:53:00)

In [ ]:
while True:
    schedule.run_pending()    
    time.sleep(1)

	Start saving 0329's data...'
195
Stored 'linklst' (list)
Show ranking 1 finished
Show ranking 2 finished
Show ranking 3 finished
Show ranking 4 finished
Show ranking 5 finished
Show ranking 6 finished
Show ranking 7 finished
Show ranking 8 finished
Show ranking 9 finished
Show ranking 10 finished
Show ranking 11 finished
Show ranking 12 finished
Show ranking 13 finished
Show ranking 14 finished
Show ranking 15 finished
Show ranking 16 finished
Show ranking 17 finished
Show ranking 18 finished
Show ranking 19 finished
Show ranking 20 finished
Show ranking 21 finished
Show ranking 22 finished
Show ranking 23 finished
Show ranking 24 finished
Show ranking 25 finished
Show ranking 26 finished
Show ranking 27 finished
Show ranking 28 finished
Show ranking 29 finished
Show ranking 30 finished
Show ranking 31 finished
Show ranking 32 finished
Show ranking 33 finished
Show ranking 34 finished
Show ranking 35 finished
Show ranking 36 finished
Show ranking 37 finished
Show ranking 38 finished
S

Stored 'linklst' (list)
